# Using FABRIC GPUs

Your compute nodes can include GPUs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use Nvidia GPU devices on FABRIC.


## Setup the Experiment

#### Import FABRIC API

In [15]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
Bastion Username,durbek_gafurov_0000000854
Bastion Private Key File,/home/fabric/work/fabric_config/bastionD
Bastion Host,bastion-1.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/.ssh/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/.ssh/slice_key
Slice Private Key Passphrase,None


In [21]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
STAR,"710 North Lake Shore Dr,Chicago, IL 60611","(41.89537135, -87.61663220067463)",6,12,384,384,0,3072,3072,0,121200,121200,0,762,762,0,2,2,0,6,6,0,20,20,0,6,6,0,6,6,0
MASS,"100 Bigelow Street,Holyoke MA 01040","(42.202493000000004, -72.60787662257826)",3,6,186,192,6,1512,1536,24,60570,60600,30,378,381,3,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
SALT,"572 Delong St,Salt Lake City, UT 84104","(40.75707505789612, -111.95346637770317)",3,6,118,192,74,1370,1536,166,60230,60600,370,354,381,27,1,2,1,2,2,0,10,10,0,2,2,0,3,3,0
NCSA,"1725 S Oak St.,Champaign, IL 61820","(40.1035624, -88.2415105)",3,6,168,192,24,1504,1536,32,60500,60600,100,378,381,3,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
UTAH,"875 South West Temple,Salt Lake City, UT 84101","(40.7626391, -111.8939563)",5,10,270,320,50,2488,2560,72,116190,116400,210,633,635,2,2,2,0,4,4,0,16,16,0,4,4,0,5,5,0
WASH,"The Bexley, 1761 Old Meadow Road, McLean, VA 22102, United States of America","(38.91930235, -77.21183383681088)",3,6,62,192,130,1344,1536,192,60110,60600,490,367,381,14,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
DALL,"1950 N Stemmons Fwy,Dallas, TX 75207","(32.8002714, -96.8198113)",3,6,158,192,34,1400,1536,136,60410,60600,190,369,381,12,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
MAX,,"(0, 0)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
MICH,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",3,6,132,192,60,1296,1536,240,59040,60600,1560,358,381,23,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
TACC,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261806879021)",5,10,234,320,86,2152,2560,408,113720,116400,2680,582,635,53,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0


## Create a Node

The cell below creates a slice that contains a single node. The node includes a GPU component.

### Set the Slice Name and FABRIC Site

Use a filter function to find random sites with your desired GPUs.


In [22]:
slice_name="CNERT 3"

rtx6000_site = "STAR" #fablib.get_random_site(filter_function=lambda x: x['rtx6000_available'] > 0) # and x['disk_available']>10000 and x['cores_available']>2 and x['ram_available']>8)

rtx6000_node_name='rtx1'
rtx6000_site

'STAR'

In [23]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    rtx_node = slice.add_node(name=rtx6000_node_name, site=rtx6000_site,disk=7000,image="default_rocky_8")
    rtx_node.add_component(model='GPU_RTX6000', name='gpu1')



    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 10, Time: 225 sec


ID,f5670179-7966-4c35-ad4b-331da370930e
Name,CNERT 3
Lease Expiration (UTC),2023-01-15 23:41:15 +0000
Lease Start (UTC),2023-01-14 23:41:15 +0000
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
7ed354dc-dc5c-4a6b-9ebb-19b391f0633e,rtx1,64,384,4000,default_rocky_8,qcow2,star-w1.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fe2a:a174,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fe2a:a174,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key



Time to stable 225 seconds
Running post_boot_config ... Time to post boot config 225 seconds


## Get the Slice

Retrieve the node information and save the management IP addresses.

In [24]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
except Exception as e:
    print(f"Exception: {e}")

ID,f5670179-7966-4c35-ad4b-331da370930e
Name,CNERT 3
Lease Expiration (UTC),2023-01-15 23:41:15 +0000
Lease Start (UTC),2023-01-14 23:41:15 +0000
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
State,StableOK


## Get the Nodes

Retrieve the nodes information and save the management IP address.


In [25]:
try:
    rtx_node = slice.get_node(rtx6000_node_name) 
    rtx_node.show()
    
    rtx_gpu = rtx_node.get_component('gpu1')
    rtx_gpu.show()
    
except Exception as e:
    print(f"Exception: {e}")

ID,7ed354dc-dc5c-4a6b-9ebb-19b391f0633e
Name,rtx1
Cores,64
RAM,384
Disk,4000
Image,default_rocky_8
Image Type,qcow2
Host,star-w1.fabric-testbed.net
Site,STAR
Username,rocky
Management IP,2001:400:a100:3030:f816:3eff:fe2a:a174


Name,rtx1-gpu1
Details,NVIDIA Corporation TU102GL [Quadro RTX 6000/8000] (rev a1)
Disk,0
Units,1
PCI Address,0000:25:00.0
Model,GPU_RTX6000
Type,GPU


Use the RTX6000 Node for the rest of the example

In [27]:
node = rtx_node


### If node has ipv6 enable it to access inet

In [30]:
from ipaddress import ip_address, IPv6Address

try:
    node = slice.get_node(name=rtx6000_node_name)     
    
    # If the node is an IPv6 Node then configure NAT64
    if type(ip_address(node.get_management_ip())) is IPv6Address:
        node.upload_file('nat64.sh', 'nat64.sh')
        
        stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
        
    # Access non-IPv6 Services
    stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')
    
    stdout, stderr = node.execute(f'ls jupyter-examples')
        
except Exception as e:
    print(f"Exception: {e}")


Installed:
  emacs-filesystem-1:26.1-7.el8.noarch                                          
  git-2.31.1-2.el8.x86_64                                                       
  git-core-2.31.1-2.el8.x86_64                                                  
  git-core-doc-2.31.1-2.el8.noarch                                              
  perl-Carp-1.42-396.el8.noarch                                                 
  perl-Data-Dumper-2.167-399.el8.x86_64                                         
  perl-Digest-1.17-395.el8.noarch                                               
  perl-Digest-MD5-2.55-396.el8.x86_64                                           
  perl-Encode-4:2.97-3.el8.x86_64                                               
  perl-Errno-1.28-421.el8.x86_64                                                
  perl-Error-1:0.17025-2.el8.noarch                                             
  perl-Exporter-5.72-396.el8.noarch                                             
  perl-File-Path

### GPU PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

View node1's GPU

In [31]:
command = "sudo dnf install -q -y pciutils && lspci | grep 'NVIDIA\|3D controller'"
try:
    stdout, stderr = node.execute(command)
except Exception as e:
    print(f"Exception: {e}")

00:07.0 3D controller: NVIDIA Corporation TU102GL [Quadro RTX 6000/8000] (rev a1)


## Install Nvidia Drivers

Now, let's run the following commands to install the latest CUDA driver and the CUDA libraries and compiler.

In [32]:
commands = [
    'sudo dnf install -q -y epel-release',
    'sudo dnf config-manager --add-repo https://developer.download.nvidia.com/compute/cuda/repos/rhel8/x86_64/cuda-rhel8.repo',
    'sudo dnf install -q -y kernel-devel kernel-headers nvidia-driver nvidia-settings cuda-driver cuda'
]
try:
    print("Installing CUDA...")
    for command in commands:
        stdout, stderr = node.execute(command)
    print("Done installing CUDA. Now, reboot for the changes to take effect.")
except Exception as e:
    print(f"Fail: {e}")

Installing CUDA...
Adding repo from: https://developer.download.nvidia.com/compute/cuda/repos/rhel8/x86_64/cuda-rhel8.repo

Upgraded:
  glibc-2.28-211.el8.x86_64           glibc-all-langpacks-2.28-211.el8.x86_64   
  glibc-common-2.28-211.el8.x86_64    libgcc-8.5.0-16.el8_7.x86_64              
  libgomp-8.5.0-16.el8_7.x86_64       libstdc++-8.5.0-16.el8_7.x86_64           
Installed:
  adwaita-cursor-theme-3.28.0-3.el8.noarch                                      
  adwaita-icon-theme-3.28.0-3.el8.noarch                                        
  alsa-lib-1.2.7.2-1.el8.x86_64                                                 
  at-spi2-atk-2.26.2-1.el8.x86_64                                               
  at-spi2-core-2.28.0-1.el8.x86_64                                              
  atk-2.28.1-1.el8.x86_64                                                       
  avahi-libs-0.7-20.el8.x86_64                                                  
  binutils-2.30-117.el8.x86_64               

In [34]:
node.execute("sudo dnf install -y make automake gcc gcc-c++ kernel-devel python3.9 ")

Last metadata expiration check: 0:02:05 ago on Sun 15 Jan 2023 12:12:22 AM UTC.
NOTE: Skipping kernel installation since no kernel module package kmod-nvidia-525.60.13-4.18.0-425.10.1 for kernel version 4.18.0-425.10.1.el8_7 and NVIDIA driver 525.60.13 could be found
Package gcc-8.5.0-16.el8_7.x86_64 is already installed.
Package gcc-c++-8.5.0-16.el8_7.x86_64 is already installed.
Package kernel-devel-4.18.0-425.10.1.el8_7.x86_64 is already installed.
Dependencies resolved.
 Package                     Arch    Version                               Repo        Size
Installing:
 automake                    noarch  1.16.1-7.el8                          appstream  712 k
 make                        x86_64  1:4.2.1-11.el8                        baseos     497 k
 python39                    x86_64  3.9.13-2.module+el8.7.0+1092+55aa9635 appstream   32 k
Installing dependencies:
 autoconf                    noarch  2.69-29.el8                           appstream  709 k
 m4                     

('Last metadata expiration check: 0:02:05 ago on Sun 15 Jan 2023 12:12:22 AM UTC.\nNOTE: Skipping kernel installation since no kernel module package kmod-nvidia-525.60.13-4.18.0-425.10.1 for kernel version 4.18.0-425.10.1.el8_7 and NVIDIA driver 525.60.13 could be found\nPackage gcc-8.5.0-16.el8_7.x86_64 is already installed.\nPackage gcc-c++-8.5.0-16.el8_7.x86_64 is already installed.\nPackage kernel-devel-4.18.0-425.10.1.el8_7.x86_64 is already installed.\nDependencies resolved.\n===========================================================================================\n Package                     Arch    Version                               Repo        Size\n===========================================================================================\nInstalling:\n automake                    noarch  1.16.1-7.el8                          appstream  712 k\n make                        x86_64  1:4.2.1-11.el8                        baseos     497 k\n python39                    x86_64

In [48]:
node.execute("pip3.9 --version")
node.execute('pip3.9 install "PyYAML>=5.3.1" "requests>=2.23.0"')

pip 20.2.4 from /usr/lib/python3.9/site-packages/pip (python 3.9)
Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.12
    Uninstalling PyYAML-3.12:
      Successfully uninstalled PyYAML-3.12
  Attempting uninstall: requests
    Found existing installation: requests 2.20.0
    Uninstalling requests-2.20.0:
      Successfully uninstalled requests-2.20.0


('Defaulting to user installation because normal site-packages is not writeable\nCollecting PyYAML>=5.3.1\n  Downloading PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)\nCollecting requests>=2.23.0\n  Downloading requests-2.28.2-py3-none-any.whl (62 kB)\nRequirement already satisfied: idna<4,>=2.5 in ./.local/lib/python3.9/site-packages (from requests>=2.23.0) (2.7)\nCollecting charset-normalizer<4,>=2\n  Downloading charset_normalizer-3.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (198 kB)\nRequirement already satisfied: certifi>=2017.4.17 in ./.local/lib/python3.9/site-packages (from requests>=2.23.0) (2022.12.7)\nRequirement already satisfied: urllib3<1.27,>=1.21.1 in ./.local/lib/python3.9/site-packages (from requests>=2.23.0) (1.24.2)\nInstalling collected packages: PyYAML, charset-normalizer, requests\n  Attempting uninstall: PyYAML\n    Found existing installation: PyYAML 3.12\n    Uninstalling P

In [42]:
node.upload_file("req.txt","req.txt")
# node.execute("pip3.9 install -r req.txt")
_,_=node.execute("cat req.txt | xargs -n 1 pip3.9 install")


Defaulting to user installation because normal site-packages is not writeable
  Using cached GitPython-3.1.18-py3-none-any.whl (170 kB)
Defaulting to user installation because normal site-packages is not writeable
  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)
Defaulting to user installation because normal site-packages is not writeable
  Using cached ipython-7.16.3-py3-none-any.whl (783 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
  Using cached jedi-0.17.2-py2.py3-none-any.whl (1.4 MB)
Defaulting to user installation because normal site-packages is not writeable
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached Jinja2-2.10.1-py2.py3-none-any.whl (124 kB)
Defaulting to user installation because normal site-packages is not writeable
  Using cached jsonpatch-1.21-py

('Defaulting to user installation because normal site-packages is not writeable\nCollecting GitPython==3.1.18\n  Using cached GitPython-3.1.18-py3-none-any.whl (170 kB)\nCollecting gitdb<5,>=4.0.1\n  Downloading gitdb-4.0.10-py3-none-any.whl (62 kB)\nCollecting smmap<6,>=3.0.1\n  Downloading smmap-5.0.0-py3-none-any.whl (24 kB)\nInstalling collected packages: smmap, gitdb, GitPython\nSuccessfully installed GitPython-3.1.18 gitdb-4.0.10 smmap-5.0.0\nDefaulting to user installation because normal site-packages is not writeable\nCollecting importlib-resources==5.4.0\n  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)\nCollecting zipp>=3.1.0; python_version < "3.10"\n  Downloading zipp-3.11.0-py3-none-any.whl (6.6 kB)\nInstalling collected packages: zipp, importlib-resources\nSuccessfully installed importlib-resources-5.4.0 zipp-3.11.0\nDefaulting to user installation because normal site-packages is not writeable\nCollecting ipython==7.16.3\n  Using cached ipython-7.16.3-py3

And once CUDA is installed, reboot the machine.

In [43]:
reboot = 'sudo reboot'
try:
    print(reboot)
    node.execute(reboot)
    
    slice.wait_ssh(timeout=360,interval=10,progress=True)

    print("Now testing SSH abilites to reconnect...",end="")
    slice.update()
    slice.test_ssh()
    print("Reconnected!")

except Exception as e:
    print(f"Fail: {e}")

sudo reboot
Waiting for slice . Slice state: StableOK
Waiting for ssh in slice ... ssh successful
Now testing SSH abilites to reconnect...Reconnected!


## Testing the GPU and CUDA Installation

First, verify that the Nvidia drivers recognize the GPU by running `nvidia-smi`.

In [44]:
try:
    stdout, stderr = node.execute("nvidia-smi")
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

Sun Jan 15 00:29:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:00:07.0 Off |                    0 |
| N/A   34C    P0    53W / 250W |      0MiB / 23040MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Now, let's upload the following "Hello World" CUDA program file to the node.

`hello-world.cu`

*Source: https://computer-graphics.se/multicore/pdf/hello-world.cu*

*Author: Ingemar Ragnemalm*

>This file is from *"The real "Hello World!" for CUDA, OpenCL and GLSL!"* (https://computer-graphics.se/hello-world-for-cuda.html), written by Ingemar Ragnemalm, programmer and CUDA teacher. The only changes (if you download the original file from the website) are to additionally `#include <unistd.h>`, as `sleep()` is now a fuction defined in the `unistd.h` library.

In [45]:
node.upload_file('./hello-world.cu', 'hello-world.cu')

<SFTPAttributes: [ size=1128 uid=1000 gid=1000 mode=0o100664 atime=1673742587 mtime=1673742587 ]>

We now compile the `.cu` file using `nvcc`, the CUDA compiler tool installed with CUDA. In this example, we create an executable called `hello_world`.

In [46]:
try:
    stdout, stderr = node.execute("/usr/local/cuda-12.0/bin/nvcc -o hello_world hello-world.cu")
except Exception as e:
    print(f"Exception: {e}")

Finally, run the executable:

In [47]:
try:
    stdout, stderr = node.execute("./hello_world")
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

Hello World!
stdout: Hello World!



If you see `Hello World!`, the CUDA program ran successfully. `World!` was computed on the GPU from an array of offsets being summed with the string `Hello `, and the resut was printed to stdout.

### Congratulations! You have now successfully run a program on a FABRIC GPU!

## Cleanup Your Experiment

In [19]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")